<a href="https://colab.research.google.com/github/mralamdari/Machine_Learning_Projects/blob/main/LanguageDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import re
import os
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


import warnings
warnings.simplefilter("ignore")

In [5]:
os.environ['KAGGLE_CONFIG_DIR']='/content/drive/MyDrive/'
!kaggle datasets download -d basilb2s/language-detection
!unzip *.zip && rm *.zip

  0% 0.00/542k [00:00<?, ?B/s]
100% 542k/542k [00:00<00:00, 77.8MB/s]
Archive:  language-detection.zip
  inflating: Language Detection.csv  


In [6]:
# Loading the dataset
data = pd.read_csv("Language Detection.csv")

In [7]:
data.head()

,Text,Language
0,"Nature, in the broadest sense, is the natural...",English
1,"""Nature"" can refer to the phenomena of the phy...",English
2,"The study of nature is a large, if not the onl...",English
3,"Although humans are part of nature, human acti...",English
4,[1] The word nature is borrowed from the Old F...,English


In [8]:
X = data["Text"]
y = data["Language"]

In [10]:
le = LabelEncoder()
y = le.fit_transform(y)

In [11]:
le.classes_

array(['Arabic', 'Danish', 'Dutch', 'English', 'French', 'German',
       'Greek', 'Hindi', 'Italian', 'Kannada', 'Malayalam', 'Portugeese',
       'Russian', 'Spanish', 'Sweedish', 'Tamil', 'Turkish'], dtype=object)

In [12]:
data_list = []
for text in X:
    text = re.sub(r'[!@#$(),\n"%^*?\:;~`0-9]', ' ', text)
    text = re.sub(r'[[]]', ' ', text)
    text = text.lower()
    data_list.append(text)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [16]:
# creating bag of words using countvectorizer
cv = CountVectorizer()
cv.fit(X_train)

x_train = cv.transform(X_train).toarray()
x_test  = cv.transform(X_test).toarray()

In [17]:
model = MultinomialNB()
model.fit(x_train, y_train)

MultinomialNB()

In [18]:
y_pred = model.predict(x_test)

In [20]:
ac = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
cr = classification_report(y_test, y_pred)

In [21]:
print("Accuracy is :",ac)

Accuracy is : 0.9763056092843327


In [22]:
pipe = Pipeline([('vectorizer', cv), ('multinomialNB', model)])
pipe.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()),
                ('multinomialNB', MultinomialNB())])

In [23]:
y_pred2 = pipe.predict(X_test)
ac2 = accuracy_score(y_test, y_pred2)
print("Accuracy is :",ac2)

Accuracy is : 0.9763056092843327


In [24]:
with open('trained_pipeline-0.1.0.pkl','wb') as f:
    pickle.dump(pipe, f)

In [25]:
# If you need to zip and download a folder, you can do it with this command:

# !zip -r ./trained_pipeline-0.1.0.pkl.zip ./trained_pipeline-0.1.0.pkl

In [26]:
text = "Hello, how are you?"
text = "Ciao, come stai?"
#text = "Hi, how are you?"
y = pipe.predict([text])
le.classes_[y[0]], y

('Italian', array([8]))